# FalkorDB Graph Store

This notebook walks through configuring `FalkorDB` to be the backend for graph storage in LlamaIndex.

In [ ]:
# My OpenAI Key
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Using Knowledge Graph with FalkorDBGraphStore

In [ ]:
from llama_index.graph_stores import FalkorDBGraphStore

graph_store = FalkorDBGraphStore("redis://localhost:6379", decode_responses=True)

#### Building the Knowledge Graph

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    KnowledgeGraphIndex,
)

from llama_index.llms import OpenAI
from IPython.display import Markdown, display

In [ ]:
documents = SimpleDirectoryReader(
    "../../../../examples/paul_graham_essay/data"
).load_data()

In [ ]:
# define LLM

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

In [ ]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)

#### Querying the Knowledge Graph

First, we can query and send only the triplets to the LLM.

In [ ]:
query_engine = index.as_query_engine(include_text=False, response_mode="tree_summarize")
response = query_engine.query(
    "Tell me more about Interleaf",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

For more detailed answers, we can also send the text from where the retrieved tripets were extracted.

In [ ]:
query_engine = index.as_query_engine(include_text=True, response_mode="tree_summarize")
response = query_engine.query(
    "Tell me more about Interleaf",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))